In [1]:
import tensorflow as tf
import numpy as np

import warnings
## Tensorflow produces a lot of warnings. We generally want to suppress them. The below code does exactly that. 
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

## Using predefined estimators
In this notebook we recreate the neural network defined first in notebook [3.Neural-Networks.ipynb](3.Neural-Networks.ipynb)  
using the predefined estimator `DNNClassifier`

## Benefits of Estimators

- Estimator-based models are independent of operating environment
    - local host
    - GPUs
    - CPU clusters

### More benefits
- Simplify model sharing between developers
- State of the art model architectures with more intuitive high-level code

## Read Data
The MNist dataset is available from within TensorFlow tutorials.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

# Wrap input as a function (THE "input function" will be defined below)
def input(dataset):
    return dataset.images, dataset.labels.astype(np.int32)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


##  Define feature columns

In [3]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

## Define Neural Network

In [4]:
# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 256],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="./tmp/mnist_model_256_256"   # Location for storing checkpoints.
)

## Define training input function
* Supplies data for training, evaluation, prediction
* Should yield tuples of:
    - Python dict `features`: key = name of feature, value = array of feature values
    - Array `label` : label for every example

In [5]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

## Train the neural network
* Checkpoint used for "warm start"
* Checkpoints saved

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)
classifier.train(input_fn=train_input_fn, steps=10000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-31000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 31000 into ./tmp/mnist_model_256_256/model.ckpt.
INFO:tensorflow:loss = 0.084843956, step = 31001
INFO:tensorflow:global_step/sec: 375.649
INFO:tensorflow:loss = 0.61875224, step = 31101 (0.267 sec)
INFO:tensorflow:global_step/sec: 448.713
INFO:tensorflow:loss = 0.73152584, step = 31201 (0.223 sec)
INFO:tensorflow:global_step/sec: 441.243
INFO:tensorflow:loss = 0.91855407, step = 31301 (

INFO:tensorflow:global_step/sec: 416.988
INFO:tensorflow:loss = 1.349119, step = 37801 (0.240 sec)
INFO:tensorflow:global_step/sec: 437.273
INFO:tensorflow:loss = 0.20243056, step = 37901 (0.228 sec)
INFO:tensorflow:global_step/sec: 429.468
INFO:tensorflow:loss = 1.0956227, step = 38001 (0.233 sec)
INFO:tensorflow:global_step/sec: 424.598
INFO:tensorflow:loss = 0.33828363, step = 38101 (0.237 sec)
INFO:tensorflow:global_step/sec: 438.412
INFO:tensorflow:loss = 0.15122673, step = 38201 (0.227 sec)
INFO:tensorflow:global_step/sec: 429.597
INFO:tensorflow:loss = 0.19989255, step = 38301 (0.233 sec)
INFO:tensorflow:global_step/sec: 432.475
INFO:tensorflow:loss = 1.3289124, step = 38401 (0.232 sec)
INFO:tensorflow:global_step/sec: 430.579
INFO:tensorflow:loss = 0.11855115, step = 38501 (0.232 sec)
INFO:tensorflow:global_step/sec: 436.143
INFO:tensorflow:loss = 1.8282694, step = 38601 (0.229 sec)
INFO:tensorflow:global_step/sec: 420.602
INFO:tensorflow:loss = 0.034283042, step = 38701 (0.237

In [7]:
# Have a look at the checkpoint directory.
!ls -lrt ./tmp/mnist_model_256_256/ | tail

-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948      16 Oct  9 21:06 model.ckpt-31000.data-00000-of-00002
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948 3231864 Oct  9 21:06 model.ckpt-31000.data-00001-of-00002
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 apache                   839 Oct  9 21:06 model.ckpt-31000.index
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948  161904 Oct  9 21:06 model.ckpt-31000.meta
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948      16 Oct  9 21:07 model.ckpt-41000.data-00000-of-00002
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948 3231864 Oct  9 21:07 model.ckpt-41000.data-00001-of-00002
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948     839 Oct  9 21:07 model.ckpt-41000.index
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948     277 Oct  9 21:07 checkpoint
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948  161904 Oct  9 21:07 model.ckpt-41000.meta
-rw-r--r-- 1 ddd_v1_t_RQDE_784948 ddd_v1_t_RQDE_784948 1078409 Oct 

## Define test input function

In [8]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False
)

## Evaluate accuracy

In [9]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-10-10-04:07:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-41000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2022-10-10-04:07:33
INFO:tensorflow:Saving dict for global step 41000: accuracy = 0.9817, average_loss = 0.074421115, global_step = 41000, loss = 9.420394
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 41000: ./tmp/mnist_model_256_256/model.ckpt-41000

Test Accuracy: 98.170000%



## Summary

- We recreated the NN defined in [3.Neural-Networks.ipynb](3.Neural-Networks.ipynb) with a higher-level API.
- The **session** was replaced with calls to the estimator.
- Checkpoints are automatically available: allows continuation of training on a partially trained NN.
- **Next:** creating custom Estimators.